#**Caso 1. Ejemplo de creación de un modelo con Máquina de Vectores de Soporte**

### Las máquinas de vector soporte o Support Vector Machines (SVM) son otro tipo de algoritmo de machine learning supervisado aplicable a problemas de regresión y clasificación, aunque se usa más comúnmente como modelo de clasificación.

# Instalación de librerías

**Instalación de librerías**
Paso 1. Revisar la versión de R instadala

In [ ]:
R.version.string

In [ ]:
install.packages("caret")
install.packages("dplyr")
install.packages("doMC")
install.packages("Boruta")

Cargando las librerías a utilizar.

In [ ]:
library(caret)
library(doMC)
library(dplyr)
library(Boruta)

# Carga de un archivo en formato CSV separado por comas para integrar el conjunto de datos a trabajar.
Este ejemplo muestra como cargar un archivo csv de un directorio local.

Primero verificamos en que directorio nos encontramos:

In [ ]:
#getwd()

Implementamos una semilla para que los datos sean siempre los mismos valores en nuestras pruebas

In [ ]:
set.seed(7)

In [ ]:

#datos <- read.csv(file = 'datav3.csv', sep = ',')
inputData <- read.csv('sample_data/datav3.csv', sep = ',') 
head(inputData)

Revisamos el contenido del dataset cargado

In [ ]:
glimpse(inputData)

In [ ]:
# Tabla de frecuencias 
table(inputData$result)

In [ ]:
# Número de observaciones del set de datos
nrow(inputData)

In [ ]:
# Detección si hay alguna fila incompleta
any(!complete.cases(inputData))

## Cambiando de tipo char a factor la clase que queremos clasificar.
Factores

Un factor es una variable categórica con un número finito de valores o niveles. En R los factores se utilizan habitualmente para realizar clasificaciones de los datos, estableciendo su pertenencia a los grupos o categorías determinados por los niveles del factor.

In [ ]:
inputData$result = as.factor(inputData$result)

In [ ]:
glimpse(inputData)

In [ ]:
#Revisando la clase
table(inputData$result)

In [ ]:
## Distribución de variables de clasificación
ggplot(data = inputData, aes(x = result, y = ..count.., fill = result)) +
  geom_bar() +
  scale_fill_manual(values = c("gray40", "orangered2")) +
  labs(title = "Clasificados con CVD") +
  theme_bw() +
  theme(legend.position = "bottom")

# Buscando los mejores atributos con Boruta

Boruta es un algoritmo envolvente de selección de características relevantes, capaz de trabajar con cualquier método de clasificación; por defecto, Boruta utiliza Random Forest. 
El método realiza una búsqueda descendente de características relevantes comparando la importancia de los atributos originales con la importancia alcanzable al azar, estimada mediante sus copias permutadas, y eliminando progresivamente las características irrelevantes para estabilizar esa prueba.

In [ ]:
# Decidir si una variable es importante o no utilizando Boruta
boruta_output <- Boruta(result ~ ., data=na.omit(inputData), doTrace=2, maxRuns=500)  # perform Boruta search
names(boruta_output)

In [ ]:
# Obtener variables significativas, incluyendo las tentativas
boruta_signif <- getSelectedAttributes(boruta_output, withTentative = TRUE)
print(boruta_signif)  

In [ ]:
# Hacer un arreglo tentativo
roughFixMod <- TentativeRoughFix(boruta_output)
print(roughFixMod)
attStats(boruta_output)
boruta_signif <- getSelectedAttributes(roughFixMod)
print(boruta_signif)


In [ ]:
# Puntuación de la importancia de las variables
imps <- attStats(roughFixMod)
imps2 = imps[imps$decision != 'Rejected', c('meanImp', 'decision')]
head(imps2[order(-imps2$meanImp), ])  # descending sort

In [ ]:
# Graficar la importancia de las variables
plot(boruta_output, cex.axis=.7, las=2, xlab="", main="Variable Importance")  

# Particionando datos para el modelo

In [ ]:
set.seed(123)

Se crea una serie de particiones de prueba/entrenamiento utilizando createDataPartition.
Argumentos:


*   y - Conjunto de datos
*   p - Porcentaje de datos a utilizar
*   list - lógico - los resultados deben estar en una lista (TRUE) o en una matriz 
*    times - El número de particiones a crear

In [ ]:
# Se crean los índices de las observaciones de entrenamiento
train <- createDataPartition(y = inputData$result, p = 0.7, list = FALSE, times = 1)
datos_train <- inputData[train, ]
datos_test  <- inputData[-train, ]

In [ ]:
# Resumen del set de datos train
glimpse(datos_train)

In [ ]:
# Resumen del set de datos train
glimpse(datos_test)

# **Creando SVM para un modelo de predicción de Taquicardia**

In [ ]:
library(doMC)
registerDoMC(cores = 4)

In [ ]:
particiones  <- 10
repeticiones <- 5

###  Valores del hiperparámetro C a evaluar y sigma

In [ ]:
# Hiperparámetros
hiperparametros <- expand.grid(sigma = c(0.001, 0.01, 0.1, 0.5, 1),
                               C = c(1 , 20, 50, 100, 200, 500, 700))

In [ ]:
set.seed(123)
seeds <- vector(mode = "list", length = (particiones * repeticiones) + 1)
for (i in 1:(particiones * repeticiones)) {
  seeds[[i]] <- sample.int(1000, nrow(hiperparametros))
}
seeds[[(particiones * repeticiones) + 1]] <- sample.int(1000, 1)

## Ajuste del modelo

In [ ]:
control_train <- trainControl(method = "repeatedcv", number = particiones,
                              repeats = repeticiones, seeds = seeds,
                              returnResamp = "all", verboseIter = FALSE,
                              classProbs = TRUE, allowParallel = TRUE)

In [ ]:
set.seed(342)

In [ ]:
install.packages("kernlab")

### Paquete caret


SVM lineal: method = “svmLinear”

SVM polinómico: method = “svmPoly”

SVM radial: method = “svmRadial”

Si no se especifica, la métrica para la evaluación es el Accuracy. Podría emplearse otra como por ejemplo “ROC”.

In [ ]:
# # Entrenamiento del SVM con un kernel radial y optimización del hiperparámetro C y sigma
svmT <- train(result ~ edad + calcio_en_sangre + taquicardia_sinusal_ecg,
              data = datos_train,
              method = "svmRadial",
              tuneGrid = hiperparametros,
              metric = "Accuracy",
              trControl = control_train)

In [ ]:
# Resultado del entrenamiento
svmT

In [ ]:
# # Entrenamiento del SVM con un kernel lineal y optimización del hiperparámetro C y sigma
svmT <- train(result ~ edad + calcio_en_sangre + taquicardia_sinusal_ecg,
              data = datos_train,
              method = "svmRadial",
              tuneGrid = hiperparametros,
              metric = "Accuracy",
              preProc = c("center", "scale"), #estandarizacion de los datos
              trControl = control_train)

In [ ]:
# Resultado del entrenamiento
svmT

In [ ]:
# Valores de validación (Accuracy y Kappa) obtenidos en cada partición y repetición.
svmT$resample %>% head(10)
summary(svmT$resample$Accuracy)

In [ ]:
# Evolución del accuracy en funcion del valor de coste en validacion cruzada
plot(svmT)

In [ ]:
ggplot(data = svmT$resample,
       aes(x = as.factor(C), y = Accuracy, color = as.factor(C))) +
  geom_boxplot(outlier.shape = NA, alpha = 0.6) +
  geom_jitter(width = 0.2, alpha = 0.6) +
  # Línea horizontal en el accuracy basal
  geom_hline(yintercept = 0.8, linetype = "dashed") +
  labs(x = "C") +
  theme_bw() + theme(legend.position = "none")

In [ ]:
ggplot(svmT, highlight = TRUE) +
  labs(title = "Evolución del accuracy del modelo en función de C") +
  theme_bw()

# **Predicciones**

In [ ]:
p <- predict(svmT, datos_test)

In [ ]:
predicciones_prob <- predict(svmT, newdata = datos_test, type = "prob")

In [ ]:
predicciones_prob

In [ ]:
predicciones_raw <- predict(svmT, newdata = datos_test, type = "raw")

In [ ]:
predicciones_raw

## EVALUACIÓN DEL MODELO

In [ ]:
###confusion Matrix
confusionMatrix(predicciones_raw, datos_test$result)

In [ ]:
# Calcular la exactitud del modelo
porcentaje <- mean(p == datos_test$result)*100
paste("Porcentaje de predicción SVM:",round(porcentaje,2))

In [ ]:
# Error de test
error_test <- mean(p != datos_test$result)
paste("El error de test del modelo SVM:", round(error_test*100, 2), "%")

In [ ]:
#Salvando el modelo
saveRDS(svmT,"svmT.rds")

# Lanzar en una API

Ejemplo de definición de API

Mostrar el modelo de taquicardia trabajando en:

http://162.214.187.236:8002/__docs__/